# LLM Optimization Modelling Experiment

In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [9]:
problem = '''A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. '''

## 2. Generate the mathematical model

In [10]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [11]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Chemical Product Production

### Sets and Indices:

*   **Products:** P = {GCA, GCB, GCC}
*   **Components:** C = {C1, C2}

### Parameters:

*   **Available component quantity:** $A_c$ (in liters) for each $c \in C$ 
    *   $A_{C1} = 10,000$
    *   $A_{C2} = 15,000$
*   **Minimum demand:** $D_p$ (in liters) for each $p \in P$
    *   $D_{GCA} = 6,000$
    *   $D_{GCB} = 7,000$
    *   $D_{GCC} = 9,000$
*   **Critical element proportion:** $E_c$ (dimensionless) for each $c \in C$
    *   $E_{C1} = 0.4$
    *   $E_{C2} = 0.2$
*   **Minimum critical element fraction in GCA:** $GCA_{min} = 0.3$
*   **Maximum critical element fraction in GCB:** $GCB_{max} = 0.3$ 
*   **Minimum C1 to C2 ratio in GCC:** $GCC_{ratio} = 0.3$
*   **Profit per liter:** $Pr_p$ (in dollars) for each $p \in P$
    *   $Pr_{GCA} = 120$
    *   $Pr_{GCB} = 135$
    *   $Pr_{GCC} = 155$ 

### Decision Variables:

*   **Liters of component c used in product p:** $x_{cp}$ for each $c \in C$ and $p \in P$

### Objective Function (Maximize Profit):

```
Maximize Z = ∑_(p∈P) ∑_(c∈C) Pr_p * x_cp
```

### Constraints:

1.  **Component Availability:** The total amount of each component used in all products cannot exceed the available quantity.

    ```
    ∑_(p∈P) x_cp ≤ A_c   ∀ c ∈ C
    ```
2.  **Minimum Demand:** The production of each product must meet the minimum demand.

    ```
    ∑_(c∈C) x_cp ≥ D_p   ∀ p ∈ P
    ```
3.  **GCA Critical Element:** The proportion of the critical element in GCA must be at least 0.3.

    ```
    (E_C1 * x_C1,GCA + E_C2 * x_C2,GCA) / (x_C1,GCA + x_C2,GCA) ≥ GCA_min
    ``` 
4.  **GCB Critical Element:** The proportion of the critical element in GCB must be at most 0.3.

    ```
    (E_C1 * x_C1,GCB + E_C2 * x_C2,GCB) / (x_C1,GCB + x_C2,GCB) ≤ GCB_max
    ```
5.  **GCC Component Ratio:** The ratio of C1 to C2 in GCC must be at least 0.3. 

    ```
    x_C1,GCC / x_C2,GCC ≥ GCC_ratio
    ```
6.  **Non-negativity:** The amount of each component used in each product cannot be negative.

    ```
    x_cp ≥ 0   ∀ c ∈ C, p ∈ P
    ```

### Model Summary:

This linear programming model aims to find the optimal production plan for the firm by maximizing profit while considering component availability, demand requirements, and critical element constraints for each product. 


## 3. Generate the pyomo code

In [20]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [21]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sets and Indices
products = ['GCA', 'GCB', 'GCC']
components = ['C1', 'C2']

# Parameters (sample data included as per problem statement)
available_quantity = {'C1': 10000, 'C2': 15000}
minimum_demand = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}
critical_element_proportion = {'C1': 0.4, 'C2': 0.2}
gca_min_critical = 0.3
gcb_max_critical = 0.3
gcc_ratio = 0.3
profit_per_liter = {'GCA': 120, 'GCB': 135, 'GCC': 155}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective Function
model.objective = pyo.Objective(
    expr=sum(profit_per_liter[p] * model.x[c, p] for c in components for p in products),
    sense=pyo.maximize
)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.x[c, p] for p in products) <= available_quantity[c]
    )

# 2. Minimum Demand
model.minimum_demand = pyo.ConstraintList()
for p in products:
    model.minimum_demand.add(
        sum(model.x[c, p] for c in components) >= minimum_demand[p]
    )

# 3. GCA Critical Element
model.gca_critical = pyo.Constraint(
    expr=(critical_element_proportion['C1'] * model.x['C1', 'GCA'] + critical_element_proportion['C2'] * model.x['C2', 'GCA']) / 
         (model.x['C1', 'GCA'] + model.x['C2', 'GCA']) >= gca_min_critical
)

# 4. GCB Critical Element
model.gcb_critical = pyo.Constraint(
    expr=(critical_element_proportion['C1'] * model.x['C1', 'GCB'] + critical_element_proportion['C2'] * model.x['C2', 'GCB']) / 
         (model.x['C1', 'GCB'] + model.x['C2', 'GCB']) <= gcb_max_critical
)

# 5. GCC Component Ratio
model.gcc_ratio = pyo.Constraint(
    expr=model.x['C1', 'GCC'] / model.x['C2', 'GCC'] >= gcc_ratio
)
```

**Explanation and Sample Data Usage:**

1.  **Sets and Indices:** We define the sets of products (`products`) and components (`components`) as per the problem statement.
2.  **Parameters:** We define the parameters with the given sample data for available quantity, minimum demand, critical element proportions, minimum/maximum critical element fractions, component ratio, and profit per liter. 
3.  **Decision Variables:** `model.x[c, p]` represents the liters of component `c` used in product `p`.  We define this as a non-negative real variable. 
4.  **Objective Function:** The objective function maximizes the total profit, which is the sum of the profit from each product (calculated as profit per liter times the quantity produced).
5.  **Constraints:**
    *   **Component Availability:** Ensures that the total usage of each component across all products doesn't exceed the available quantity.
    *   **Minimum Demand:** Guarantees that the production of each product meets its minimum demand requirement.
    *   **GCA/GCB Critical Element:** Enforces the constraints on the critical element proportions in GCA and GCB. 
    *   **GCC Component Ratio:** Ensures the required ratio of C1 to C2 in GCC.

**Running the Model:**

You would typically use a solver like `pyomo.opt.SolverFactory('glpk')` to solve this model and obtain the optimal production quantities for each product. 


## 4. Input problem data and try running the generated code

In [24]:
import pyomo.environ as pyo

# Sets and Indices
products = ['GCA', 'GCB', 'GCC']
components = ['C1', 'C2']

# Parameters (sample data included as per problem statement)
available_quantity = {'C1': 10000, 'C2': 15000}
minimum_demand = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}
critical_element_proportion = {'C1': 0.4, 'C2': 0.2}
gca_min_critical = 0.3
gcb_max_critical = 0.3
gcc_ratio = 0.3
profit_per_liter = {'GCA': 120, 'GCB': 135, 'GCC': 155}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective Function
model.objective = pyo.Objective(
    expr=sum(profit_per_liter[p] * model.x[c, p] for c in components for p in products),
    sense=pyo.maximize
)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.x[c, p] for p in products) <= available_quantity[c]
    )

# 2. Minimum Demand
model.minimum_demand = pyo.ConstraintList()
for p in products:
    model.minimum_demand.add(
        sum(model.x[c, p] for c in components) >= minimum_demand[p]
    )

# 3. GCA Critical Element
model.gca_critical = pyo.Constraint(
    expr=(critical_element_proportion['C1'] * model.x['C1', 'GCA'] + critical_element_proportion['C2'] * model.x['C2', 'GCA']) / 
         (model.x['C1', 'GCA'] + model.x['C2', 'GCA']) >= gca_min_critical
)

# 4. GCB Critical Element
model.gcb_critical = pyo.Constraint(
    expr=(critical_element_proportion['C1'] * model.x['C1', 'GCB'] + critical_element_proportion['C2'] * model.x['C2', 'GCB']) / 
         (model.x['C1', 'GCB'] + model.x['C2', 'GCB']) <= gcb_max_critical
)

# 5. GCC Component Ratio
model.gcc_ratio = pyo.Constraint(
    expr=model.x['C1', 'GCC'] / model.x['C2', 'GCC'] >= gcc_ratio
)

# Solve the model
solver = SolverFactory('ipopt')  # You can use any LP solver available   #ALTERED BY A HUMAN
solver.solve(model)

# Print the optimal solution
print("Optimal Solution:")
print(f"Total Profit = {model.objective():.2f}")


Optimal Solution:
Total Profit = 3525000.04


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [25]:
print(response.text)

## Mathematical Optimization Model for Chemical Product Production

### Sets and Indices:

*   **Products:** P = {GCA, GCB, GCC}
*   **Components:** C = {C1, C2}

### Parameters:

*   **Available component quantity:** $A_c$ (in liters) for each $c \in C$ 
    *   $A_{C1} = 10,000$
    *   $A_{C2} = 15,000$
*   **Minimum demand:** $D_p$ (in liters) for each $p \in P$
    *   $D_{GCA} = 6,000$
    *   $D_{GCB} = 7,000$
    *   $D_{GCC} = 9,000$
*   **Critical element proportion:** $E_c$ (dimensionless) for each $c \in C$
    *   $E_{C1} = 0.4$
    *   $E_{C2} = 0.2$
*   **Minimum critical element fraction in GCA:** $GCA_{min} = 0.3$
*   **Maximum critical element fraction in GCB:** $GCB_{max} = 0.3$ 
*   **Minimum C1 to C2 ratio in GCC:** $GCC_{ratio} = 0.3$
*   **Profit per liter:** $Pr_p$ (in dollars) for each $p \in P$
    *   $Pr_{GCA} = 120$
    *   $Pr_{GCB} = 135$
    *   $Pr_{GCC} = 155$ 

### Decision Variables:

*   **Liters of component c used in product p:** $x_{cp}$ for each

In [26]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sets and Indices
products = ['GCA', 'GCB', 'GCC']
components = ['C1', 'C2']

# Parameters (sample data included as per problem statement)
available_quantity = {'C1': 10000, 'C2': 15000}
minimum_demand = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}
critical_element_proportion = {'C1': 0.4, 'C2': 0.2}
gca_min_critical = 0.3
gcb_max_critical = 0.3
gcc_ratio = 0.3
profit_per_liter = {'GCA': 120, 'GCB': 135, 'GCC': 155}

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.x = pyo.Var(components, products, domain=pyo.NonNegativeReals)

# Objective Function
model.objective = pyo.Objective(
    expr=sum(profit_per_liter[p] * model.x[c, p] for c in components for p in products),
    sense=pyo.maximize
)

# Constraints
# 1. Component Availability
model.component_availability = pyo.ConstraintList()
for c in components:
    model.component_availability.add(
        sum(model.x[c, p] for p in products) <= available_quantity[c]
    )

# 2. Min